In [55]:
evals_record_paths = ['/home/zaid/.evals/eval_results/ajgt2.jsonl', '/home/zaid/.evals/eval_results/ajgt2.jsonl_resume']

In [98]:
import json


metrics = {'classification':['accuracy']}

def merge_evals(task_class, eval_name, evals_record_paths =[], max_samples = -1, ):
    out_records = []
    curr_sample_id = 0
    curr_event_id = 0 
    for record_path in evals_record_paths:
        with open(record_path, "r") as records_file:
            records_str = records_file.read().splitlines()
            records = [json.loads(record_str) for record_str in records_str]
            records = list(filter(lambda record: "type" in record.keys(), records)) #extract records with types
            task_records = list(filter(lambda record: record["type"] == task_class, records)) #extract records with types
            for i in range(len(task_records)):
                # filter the records containing the sample id
                sample_id_records = list(filter(lambda record: record["sample_id"] == f'{eval_name}.test.{i}', records))
                # print(sample_id_records)
                sample_id_record_copy = {}
                for sample_id_record in sample_id_records:
                    sample_id_record_copy = dict(sample_id_record) # don't modify a dict inside a for loop 
                    record_id = int(sample_id_record['sample_id'].split('.')[-1])
                    sample_id_record_copy['sample_id'] = f"{eval_name}.test.{curr_sample_id}"
                    sample_id_record_copy['event_id'] = curr_event_id
                    curr_event_id += 1
                    out_records.append(sample_id_record_copy)
                curr_sample_id += 1

    final_report =  {metric: 0.0 for metric in metrics[task_class]}
    task_metrics = metrics[task_class]
    for record in out_records:
        if record['type'] == task_class:
            for metric in task_metrics:
                final_report[metric] += record["data"][metric] / max_samples # what happens if there are repeated samples ?:

    out_records.append({'final_report':final_report})
    return out_records

In [100]:
merge_evals (eval_name = "ajgt2", task_class = "classification", max_samples= 3, evals_record_paths = evals_record_paths)

/home/zaid/.evals/eval_results/ajgt2.jsonl
/home/zaid/.evals/eval_results/ajgt2.jsonl_resume


[{'run_id': '23060307235557BURB5O',
  'event_id': 0,
  'sample_id': 'ajgt2.test.0',
  'type': 'raw_sample',
  'data': {'input': [{'role': 'system', 'content': 'What is the sentiment?'},
    {'role': 'user',
     'content': ' اربد فيها جامعات اكثر من عمان ... وفيها قد عمان ونص لعيبه المنتخب منها ... و 80 % من مطربين الاردن منها'}],
   'ideal': 'Positive'},
  'created_by': '',
  'created_at': '2023-06-03 07:23:55.954976+00:00'},
 {'run_id': '23060307235557BURB5O',
  'event_id': 1,
  'sample_id': 'ajgt2.test.0',
  'type': 'sampling',
  'data': {'prompt': [{'role': 'system', 'content': 'What is the sentiment?'},
    {'role': 'user',
     'content': ' اربد فيها جامعات اكثر من عمان ... وفيها قد عمان ونص لعيبه المنتخب منها ... و 80 % من مطربين الاردن منها'}],
   'sampled': 'The sentiment in this statement is positive towards Irbid, as it highlights its academic and cultural significance.',
   'metadata': {'completion_id': 'chatcmpl-7NFu7YUofB1JD747e2WV5LM0tpUVu',
    'model': 'gpt-3.5-turbo-0

In [95]:
for record in out_records:
    if 'type' in record and  record['type'] == 'raw_sample':
        print(record['data']['input'][1])

{'role': 'user', 'content': ' اربد فيها جامعات اكثر من عمان ... وفيها قد عمان ونص لعيبه المنتخب منها ... و 80 % من مطربين الاردن منها'}
{'role': 'user', 'content': ' الحلو انكم بتحكوا على اساس انو الاردن ما فيه فساد سرقات'}
{'role': 'user', 'content': ' كله رائع بجد ربنا يكرمك'}


In [96]:
from datasets import load_dataset
dataset = load_dataset('ajgt_twitter_ar')

Found cached dataset ajgt_twitter_ar (/home/zaid/.cache/huggingface/datasets/ajgt_twitter_ar/plain_text/1.0.0/52248534d006e64154333ee6bb8bc3bea8545e00cfc156eaba190bf52dcff6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

In [97]:
for i, sample in enumerate(dataset['train']):
    print(sample)
    if i >= 2:
        break

{'text': ' اربد فيها جامعات اكثر من عمان ... وفيها قد عمان ونص لعيبه المنتخب منها ... و 80 % من مطربين الاردن منها', 'label': 1}
{'text': ' الحلو انكم بتحكوا على اساس انو الاردن ما فيه فساد سرقات', 'label': 0}
{'text': ' كله رائع بجد ربنا يكرمك', 'label': 1}
